In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/andyhftang/aasd4010-project/main/model/stock_data.csv?token=GHSAT0AAAAAACS52BKUFG4KSGLB5IKDQJGOZSZARHA'
data = pd.read_csv('stock_data.csv')


# Step 1: Data Preprocessing

In [2]:
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Clean the text data
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

data['cleaned_text'] = data['Text'].apply(clean_text)

# Tokenize the text
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(data['cleaned_text'])
sequences = tokenizer.texts_to_sequences(data['cleaned_text'])
padded_sequences = pad_sequences(sequences, maxlen=50, padding='post')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, data['Sentiment'], test_size=0.2, random_state=42)


# Step 2: Model Selection

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=50),
    Bidirectional(LSTM(64, dropout=0.2, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 64)            320000    
                                                                 
 bidirectional (Bidirection  (None, 50, 128)           66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 427329 (1.63 MB)
Trainable params: 427329 (1.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Step 3: Model Training

In [4]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=64)


Epoch 1/20
73/73 [==============================] - 24s 201ms/step - loss: 0.5136 - accuracy: 0.0214 - val_loss: -0.1154 - val_accuracy: 0.2942
Epoch 2/20
73/73 [==============================] - 13s 174ms/step - loss: -1.5470 - accuracy: 0.4763 - val_loss: -1.4184 - val_accuracy: 0.4806
Epoch 3/20
73/73 [==============================] - 13s 174ms/step - loss: -3.6694 - accuracy: 0.5006 - val_loss: -2.4884 - val_accuracy: 0.4288
Epoch 4/20
73/73 [==============================] - 13s 172ms/step - loss: -5.9622 - accuracy: 0.5300 - val_loss: -3.1232 - val_accuracy: 0.5298
Epoch 5/20
73/73 [==============================] - 11s 144ms/step - loss: -7.8418 - accuracy: 0.5468 - val_loss: -3.9252 - val_accuracy: 0.4668
Epoch 6/20
73/73 [==============================] - 18s 247ms/step - loss: -8.7718 - accuracy: 0.5475 - val_loss: -4.8239 - val_accuracy: 0.4625
Epoch 7/20
73/73 [==============================] - 10s 141ms/step - loss: -10.4549 - accuracy: 0.5579 - val_loss: -4.7657 - val_ac

# Step 4: Model Evaluation

In [5]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


37/37 [==============================] - 1s 26ms/step - loss: -14.3414 - accuracy: 0.4814
Loss: -14.341419219970703, Accuracy: 0.48144951462745667


# Step 5: Model Export

In [10]:
import pickle

# Save the model
model.save('semantic_analysis_model.h5')

# Save the Tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)



/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Step 6: Prediction Script

In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the trained model
model = load_model('semantic_analysis_model.h5')

# Load the Tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Function to predict text
def predict_text(text):
    cleaned_text = clean_text(text)
    sequence = tokenizer.texts_to_sequences([cleaned_text])
    padded_sequence = pad_sequences(sequence, maxlen=50, padding='post')
    prediction = model.predict(padded_sequence)
    return 'Positive' if prediction >= 0.5 else 'Negative'


In [12]:
# Example usage
new_text = "Don't buy stock today!!"
prediction = predict_text(new_text)
print(f'The sentiment of the news headline "{new_text}" is {prediction}.')


# Example usage
new_text = "Today is the best day to buy stock!"
prediction = predict_text(new_text)
print(f'The sentiment of the news headline "{new_text}" is {prediction}.')

1/1 [==============================] - 2s 2s/step
The sentiment of the news headline "Don't buy stock today!!" is Negative.
1/1 [==============================] - 0s 29ms/step
The sentiment of the news headline "Today is the best day to buy stock!" is Positive.
